In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gemma-3n/transformers/gemma-3n-e2b-it/1/model.safetensors.index.json
/kaggle/input/gemma-3n/transformers/gemma-3n-e2b-it/1/model-00003-of-00003.safetensors
/kaggle/input/gemma-3n/transformers/gemma-3n-e2b-it/1/config.json
/kaggle/input/gemma-3n/transformers/gemma-3n-e2b-it/1/preprocessor_config.json
/kaggle/input/gemma-3n/transformers/gemma-3n-e2b-it/1/README.md
/kaggle/input/gemma-3n/transformers/gemma-3n-e2b-it/1/tokenizer.json
/kaggle/input/gemma-3n/transformers/gemma-3n-e2b-it/1/model-00001-of-00003.safetensors
/kaggle/input/gemma-3n/transformers/gemma-3n-e2b-it/1/tokenizer_config.json
/kaggle/input/gemma-3n/transformers/gemma-3n-e2b-it/1/chat_template.jinja
/kaggle/input/gemma-3n/transformers/gemma-3n-e2b-it/1/model-00002-of-00003.safetensors
/kaggle/input/gemma-3n/transformers/gemma-3n-e2b-it/1/processor_config.json
/kaggle/input/gemma-3n/transformers/gemma-3n-e2b-it/1/special_tokens_map.json
/kaggle/input/gemma-3n/transformers/gemma-3n-e2b-it/1/tokenizer.model
/kag

In [2]:
# ==========================
# Agritech Advisor Demo with Gemma 3n (E2B-IT) - Enhanced Version
# ==========================
"""
Enhanced Agritech Advisor powered by Gemma 3n multimodal AI model.

This notebook demonstrates a comprehensive agricultural advisory system that:
- Processes multimodal inputs (text, image, audio)
- Provides AI-powered crop health analysis
- Offers actionable agricultural recommendations
- Supports up to 32K input/output tokens
- Works offline once models are downloaded

Author: DataZimbo
Date: 2025-07-29
"""

# --------
# 1. Install Dependencies
# --------
# Install required packages for multimodal AI processing
# - timm: PyTorch Image Models for advanced computer vision
# - accelerate: Hugging Face library for distributed/mixed precision training
# - transformers: State-of-the-art NLP and multimodal models
# - kagglehub: Official Kaggle API for downloading datasets and models
!pip install -q --upgrade timm accelerate git+https://github.com/huggingface/transformers.git kagglehub

# --------
# 2. Imports and Setup
# --------
import os              # Operating system interface for environment variables
import gc              # Garbage collector for memory management
import re              # Regular expressions for text processing
import json            # JSON handling for structured data
import warnings        # Warning control for cleaner output
import torch           # PyTorch deep learning framework
from PIL import Image  # Python Imaging Library for image processing
from io import BytesIO # In-memory binary streams for image handling
from IPython.display import display, HTML  # Jupyter notebook display utilities
import base64          # Base64 encoding for image display
import requests        # HTTP requests for fetching images from URLs
import kagglehub       # Kaggle Hub API for model downloading
from transformers import AutoModelForImageTextToText, AutoProcessor  # Hugging Face transformers

# --------
# 3. Environment and Device Configuration
# --------
# Suppress unnecessary warnings for cleaner output
warnings.filterwarnings("ignore", category=UserWarning)

# Disable PyTorch optimizations that might cause issues in notebook environments
os.environ["TORCHINDUCTOR_DISABLE"] = "1"  # Disable TorchInductor compilation
os.environ["DISABLE_TORCH_COMPILE"] = "1"   # Disable torch.compile optimization

def get_device():
    """
    Automatically detect and return the best available compute device.
    
    Returns:
        torch.device: CUDA GPU if available, otherwise CPU
        
    Note:
        GPU acceleration significantly improves inference speed for large models
    """
    return torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize device and display current configuration
device = get_device()
print(f'Using device: {device}')
if device.type == 'cuda':
    print(f'GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB')

# --------
# 4. KaggleHub Authentication and Model Download (Gemma 3n E2B-IT)
# --------
# Define Gemma 3n model resource configuration
# E2B-IT stands for "End-to-End Bilingual Instruction Tuned"
GEMMA_MODEL_RESOURCE = {
    "gemma_e2b_it": {
        "model_download": "google/gemma-3n/Transformers/gemma-3n-e2b-it/1"
    }
}

def authenticate_and_download_kaggle_resources(resource_dict):
    """
    Authenticate with Kaggle and download specified resources.
    
    Args:
        resource_dict (dict): Dictionary containing resource configurations
                             with 'competition_download' or 'model_download' keys
    
    Returns:
        dict: Local file paths for downloaded resources
        
    Raises:
        ValueError: If unknown resource type is encountered
        
    Note:
        Requires Kaggle API credentials to be configured
        (kaggle.json in ~/.kaggle/ or environment variables)
    """
    # Authenticate with Kaggle Hub using stored credentials
    kagglehub.login()
    
    local_paths = {}
    for name, resource in resource_dict.items():
        print(f"Downloading {name}...")
        
        if 'competition_download' in resource:
            # Download competition dataset
            local_paths[name] = kagglehub.competition_download(resource['competition_download'])
        elif 'model_download' in resource:
            # Download pre-trained model
            local_paths[name] = kagglehub.model_download(resource['model_download'])
        else:
            raise ValueError(f"Unknown resource type for {name}: {resource}")
            
        print(f"✓ {name} downloaded to: {local_paths[name]}")
    
    return local_paths

# Download Gemma 3n model and get local path
print("Authenticating and downloading Gemma 3n model...")
RESOURCES = authenticate_and_download_kaggle_resources(GEMMA_MODEL_RESOURCE)
GEMMA_E2B_IT_PATH = RESOURCES["gemma_e2b_it"]
print(f"Model ready at: {GEMMA_E2B_IT_PATH}")

# --------
# 5. Model and Processor Loading
# --------
def load_hf_model_and_processor(model_path, model_class=AutoModelForImageTextToText, 
                               processor_class=AutoProcessor, device=None):
    """
    Load Hugging Face model and processor from local path.
    
    Args:
        model_path (str): Local path to the downloaded model
        model_class: Hugging Face model class (default: AutoModelForImageTextToText)
        processor_class: Hugging Face processor class (default: AutoProcessor)
        device: Target device for model inference
    
    Returns:
        tuple: (model, processor) ready for inference
        
    Note:
        - Uses automatic mixed precision (torch_dtype="auto") for efficiency
        - Loads model to specified device for faster inference
    """
    if device is None:
        device = get_device()
    
    print("Loading model and processor...")
    
    # Load the multimodal model with automatic precision
    model = model_class.from_pretrained(
        model_path, 
        device_map=None,           # Manual device placement
        torch_dtype="auto"         # Automatic precision selection
    ).to(device)
    
    # Load the corresponding processor for input preprocessing
    processor = processor_class.from_pretrained(model_path)
    
    print(f"✓ Model loaded on {device}")
    print(f"✓ Model parameters: ~{sum(p.numel() for p in model.parameters()) / 1e9:.1f}B")
    
    return model, processor

# Load Gemma 3n model and processor
model, processor = load_hf_model_and_processor(GEMMA_E2B_IT_PATH, device=device)

# --------
# 6. Input Utilities (Text, Image, Audio)
# --------
def detect_input_type(input_data):
    """
    Automatically detect input type based on data characteristics.
    
    Args:
        input_data (str): Input data (file path, URL, or text)
    
    Returns:
        str: "image", "audio", or "text"
        
    Examples:
        >>> detect_input_type("photo.jpg")
        "image"
        >>> detect_input_type("recording.wav")
        "audio"
        >>> detect_input_type("My crops are wilting")
        "text"
    """
    if isinstance(input_data, str):
        # Check for image file extensions
        if input_data.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff")):
            return "image"
        # Check for audio file extensions
        elif input_data.lower().endswith((".wav", ".mp3", ".m4a", ".ogg", ".flac")):
            return "audio"
    
    # Default to text for everything else
    return "text"

def fetch_image(raw_input):
    """
    Fetch and load image from URL or local file path.
    
    Args:
        raw_input (str): Image URL or local file path
    
    Returns:
        PIL.Image: RGB image ready for processing
        
    Raises:
        ValueError: If image cannot be loaded or accessed
        
    Note:
        Automatically converts images to RGB format for consistency
    """
    try:
        if raw_input.startswith("http://") or raw_input.startswith("https://"):
            # Fetch image from URL
            print(f"Fetching image from URL: {raw_input}")
            response = requests.get(raw_input, timeout=30)
            response.raise_for_status()  # Raise exception for bad status codes
            image = Image.open(BytesIO(response.content)).convert("RGB")
        else:
            # Load image from local file
            print(f"Loading image from file: {raw_input}")
            image = Image.open(raw_input).convert("RGB")
        
        print(f"✓ Image loaded: {image.size[0]}x{image.size[1]} pixels")
        return image
        
    except Exception as e:
        raise ValueError(f"Failed to load image from {raw_input}: {e}")

def normalize_image(image, target_size=512):
    """
    Normalize image to supported dimensions for Gemma 3n processing.
    
    Args:
        image (PIL.Image): Input image to normalize
        target_size (int): Desired output size (default: 512)
    
    Returns:
        PIL.Image: Resized image with dimensions from [256, 512, 768]
        
    Note:
        Gemma 3n supports specific image sizes for optimal performance.
        The function selects the closest supported size to the target.
    """
    # Gemma 3n supported image dimensions
    allowed_sizes = [256, 512, 768]
    
    # Select closest allowed size to target
    if isinstance(target_size, int):
        size = min(allowed_sizes, key=lambda x: abs(x - target_size))
    else:
        size = 512  # Default fallback
    
    print(f"Normalizing image to {size}x{size} pixels")
    normalized_image = image.resize((size, size), Image.Resampling.LANCZOS)
    
    return normalized_image

def display_resized_image(image, width_px=300):
    """
    Display PIL image in Jupyter notebook with specified width.
    
    Args:
        image (PIL.Image): Image to display
        width_px (int): Display width in pixels (default: 300)
        
    Note:
        Converts image to base64 for HTML embedding in notebook
    """
    # Convert PIL image to base64 for HTML display
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    img_b64 = base64.b64encode(buffered.getvalue()).decode("utf-8")
    
    # Create HTML img tag with base64 data
    html = f'<img src="data:image/png;base64,{img_b64}" width="{width_px}px" style="border-radius: 8px;"/>'
    display(HTML(html))

def format_input(input_type, raw_input):
    """
    Process and format input data based on detected type.
    
    Args:
        input_type (str): Type of input ("image", "audio", or "text")
        raw_input: Raw input data (URL, file path, or text string)
    
    Returns:
        Formatted input ready for model processing
        
    Note:
        - Images are normalized to supported dimensions
        - Audio is converted to text via simulated ASR (placeholder)
        - Text is passed through unchanged
    """
    if input_type == "image":
        # Process image input
        image = fetch_image(raw_input)
        image = normalize_image(image, target_size=512)
        
        print("📸 Image processed successfully. Preview:")
        display_resized_image(image, width_px=300)
        return image
        
    elif input_type == "audio":
        # Simulate audio-to-text conversion
        # In production, this would use a speech recognition model
        simulated_transcript = "Leaves turning yellow and curling at the edges."
        print(f"🎤 Audio input detected (simulated)")
        print(f"   Transcript: '{simulated_transcript}'")
        print("   Note: In production, this would use actual ASR processing")
        return simulated_transcript
        
    elif input_type == "text":
        # Text input passes through unchanged
        print(f"📝 Text input: '{raw_input[:100]}{'...' if len(raw_input) > 100 else ''}'")
        return raw_input
        
    else:
        raise ValueError(f"Unsupported input type: {input_type}")

# --------
# 7. Agritech Instructions and Prompt Engineering
# --------
AGRITECH_ADVISOR_INSTRUCTIONS = """
You are an expert Agritech Advisor powered by Gemma 3n multimodal AI.

Your role is to analyze agricultural inputs and provide actionable advice to farmers.
You can process images of crops, transcribed audio questions, or written text queries.

ANALYSIS WORKFLOW:
1. **Input Detection**: Identify if input is visual (crop image) or textual (question/description)

2. **Image Analysis** (if applicable):
   - Examine crop health indicators (leaf color, texture, growth patterns)
   - Identify potential diseases, pests, or nutrient deficiencies
   - Assess environmental stress signs (drought, overwatering, etc.)
   - Note growth stage and overall plant condition

3. **Problem Diagnosis**:
   - Classify issues: disease, pest, nutrient deficiency, environmental stress
   - Determine severity level and urgency of intervention
   - Consider regional context if provided (climate, season, location)

4. **Recommendation Generation**:
   - Provide specific, actionable solutions
   - Include both immediate and long-term strategies
   - Consider resource availability and farmer accessibility
   - Prioritize sustainable and cost-effective approaches

RESPONSE FORMAT:
Always respond with properly formatted JSON enclosed in triple backticks:

```json
{
  "input_type": "<image/speech/text>",
  "analysis": "<detailed observation of input or visual analysis>",
  "diagnosis": "<specific issue identified or 'healthy' if no problems>",
  "confidence": "<high/medium/low based on analysis certainty>",
  "advice": "<primary actionable recommendation>",
  "urgency": "<low/medium/high based on problem severity>",
  "tips": ["<specific tip 1>", "<specific tip 2>", "<specific tip 3>"],
  "timeline": "<immediate/within days/within weeks - when to act>",
  "prevention": "<how to prevent this issue in future>"
} """

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.1/367.1 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 26.6 MB/s 

2025-07-29 15:36:29.513622: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753803389.854234      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753803389.950381      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using device: cuda
GPU Memory: 14.7 GB
Authenticating and downloading Gemma 3n model...


✓ gemma_e2b_it downloaded to: /kaggle/input/gemma-3n/transformers/gemma-3n-e2b-it/1
Model ready at: /kaggle/input/gemma-3n/transformers/gemma-3n-e2b-it/1
Loading model and processor...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some weights of Gemma3nForConditionalGeneration were not initialized from the model checkpoint at /kaggle/input/gemma-3n/transformers/gemma-3n-e2b-it/1 and are newly initialized: ['model.vision_tower.timm_model.conv_stem.conv.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✓ Model loaded on cuda
✓ Model parameters: ~5.4B


In [3]:
def run_agritech_advisor(input_data, context_hint="", model=None, processor=None, device=None, max_tokens=2048):
    """Main function to run the Agritech Advisor on various input types."""
    print("🌱 Starting Agritech Advisor Analysis...")
    print("=" * 50)
    
    if device is None:
        device = get_device()
    if model is None or processor is None:
        raise ValueError("Model and processor must be provided")
    
    try:
        # Detect and process input type
        input_type = detect_input_type(input_data)
        print(f"📋 Input type detected: {input_type.upper()}")
        
        # Format input data
        formatted_input = format_input(input_type, input_data)
        
        # Create prompt
        prompt = f"""{AGRITECH_ADVISOR_INSTRUCTIONS}
CONTEXT: {context_hint}
ANALYZE THE FOLLOWING INPUT:
"""
        
        # Prepare inputs for model
        print("\n🔄 Processing with Gemma 3n...")
        
        if input_type == "image":
            inputs = processor(text=prompt, images=formatted_input, return_tensors="pt").to(device)
        else:
            full_text = prompt + str(formatted_input)
            inputs = processor(text=full_text, return_tensors="pt").to(device)
        
        # Generate response
        print("⚡ Generating AI response...")
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_tokens,
                do_sample=True,
                temperature=0.3,
                top_p=0.9,
                pad_token_id=processor.tokenizer.eos_token_id
            )
        
        # Decode response
        response = processor.decode(outputs[0], skip_special_tokens=True)
        print("✅ Response generated!")
        print("\nRaw response:")
        print(response)
        
        return {"response": response}
        
    except Exception as e:
        print(f"❌ Error: {e}")
        return {"error": str(e)}

# Now you can run your example
result = run_agritech_advisor(
    input_data="My tomato plants have yellow leaves with brown spots. What should I do?",
    context_hint="Tomatoes, Kenya, dry season",
    model=model,
    processor=processor,
    device=device
)

🌱 Starting Agritech Advisor Analysis...
📋 Input type detected: TEXT
📝 Text input: 'My tomato plants have yellow leaves with brown spots. What should I do?'

🔄 Processing with Gemma 3n...
⚡ Generating AI response...


W0729 15:40:33.382000 19 torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode


❌ Error: generator


In [4]:
# Run a text-based agricultural question
result = run_agritech_advisor(
    input_data="My tomato plants have yellow leaves with brown spots. What should I do?",
    context_hint="Tomatoes, Kenya, dry season",
    model=model,
    processor=processor,
    device=device
)


🌱 Starting Agritech Advisor Analysis...
📋 Input type detected: TEXT
📝 Text input: 'My tomato plants have yellow leaves with brown spots. What should I do?'

🔄 Processing with Gemma 3n...
⚡ Generating AI response...
❌ Error: generator
